# TDSP Stage 2 – Data Acquisition & Understanding

## Proyek: Segmentasi Pelanggan Online Retail Berbasis RFM (Spark)

Stage ini berfokus pada proses **pengambilan data**, **pemahaman struktur data**, serta **eksplorasi awal** menggunakan Apache Spark.
Dataset yang digunakan berada dalam format **Parquet**, sehingga efisien untuk diproses secara terdistribusi.

## 2.1 Deskripsi Dataset

Dataset berisi **data transaksi online retail** dengan karakteristik utama:

- Setiap baris merepresentasikan **item produk** dalam suatu transaksi
- Satu transaksi dapat terdiri dari banyak item
- Pelanggan diidentifikasi menggunakan **PostCode** sebagai *proxy customer ID*

Kolom utama yang relevan untuk analisis RFM:
- `Invoice`
- `StockCode`
- `Quantity`
- `Price`
- `InvoiceDate`
- `PostCode`
- `Country`


In [ ]:
# ==============================================================
# Inisialisasi Spark Session
# ==============================================================
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("RFM-Customer-Segmentation")
    .getOrCreate()
)

spark

## 2.2 Load Data dari Parquet

Data transaksi disimpan dalam format **Parquet** pada data lake / storage.
Format ini dipilih karena:
- Columnar storage (lebih cepat untuk agregasi)
- Mendukung schema evolution
- Sangat optimal untuk Spark


In [ ]:
# ==============================================================
# Load dataset transaksi dari Parquet
# ==============================================================
# Catatan:
# - Sesuaikan path dengan environment (local / Fabric / Databricks)

DATA_PATH = "/data/online_retail/transactions.parquet"

df_raw = spark.read.parquet(DATA_PATH)

df_raw.printSchema()
df_raw.show(5, truncate=False)

## 2.3 Pemahaman Skema & Struktur Data

Langkah awal yang penting adalah memahami:
- Tipe data setiap kolom
- Apakah terdapat kolom yang tidak relevan
- Konsistensi format tanggal dan numerik

Pemahaman ini akan sangat mempengaruhi proses **feature engineering RFM** pada stage berikutnya.

In [ ]:
# ==============================================================
# Statistik dasar & jumlah baris
# ==============================================================
from pyspark.sql.functions import count

df_raw.select([count(c).alias(c) for c in df_raw.columns]).show(truncate=False)

print(f"Jumlah baris data: {df_raw.count()}")

## 2.4 Pemeriksaan Kualitas Data (Data Quality Check)

Beberapa aspek kualitas data yang perlu diperiksa:
- Nilai kosong (null) pada kolom penting
- Transaksi tanpa PostCode
- Transaksi dari negara di luar scope analisis

Tahap ini bersifat **diagnostik**, belum melakukan pembersihan permanen.

In [ ]:
# ==============================================================
# Cek jumlah nilai null per kolom
# ==============================================================
from pyspark.sql.functions import col, sum as spark_sum

null_summary = df_raw.select([
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in df_raw.columns
])

null_summary.show(truncate=False)

## 2.5 Filter Awal Berdasarkan Kebutuhan Bisnis

Sesuai dengan ruang lingkup proyek:
- Hanya transaksi dengan **PostCode valid** yang dipertahankan
- Analisis difokuskan pada **pelanggan dari UK**

Filter ini bersifat **sementara**, hasil akhirnya akan digunakan pada TDSP Stage 3.

In [ ]:
# ==============================================================
# Filter transaksi valid (PostCode tidak null & UK only)
# ==============================================================
df_filtered = (
    df_raw
    .filter(col("PostCode").isNotNull())
    .filter(col("Country") == "United Kingdom")
)

print(f"Jumlah baris setelah filter awal: {df_filtered.count()}")

## 2.6 Insight Awal (Initial Observations)

Beberapa insight awal yang biasanya muncul pada tahap ini:
- Distribusi jumlah item per transaksi cenderung tidak merata
- Sebagian kecil pelanggan berkontribusi pada volume transaksi yang besar
- Terdapat indikasi pelanggan organisasi (B2B) dengan quantity tinggi

Insight ini akan diperdalam melalui perhitungan **Recency, Frequency, dan Monetary** pada stage berikutnya.

## Ringkasan Stage 2

Pada TDSP Stage 2 ini, kita telah:
- Memuat data transaksi dari format Parquet menggunakan Spark
- Memahami struktur dan skema data
- Melakukan pemeriksaan kualitas data awal
- Menerapkan filter dasar sesuai kebutuhan bisnis

Tahap selanjutnya adalah **TDSP Stage 3 – Modeling**, yang mencakup:
- Feature engineering RFM
- Normalisasi data
- Clustering pelanggan berbasis Spark